In [7]:
from PIL import Image 
from IPython.display import display 
import random
import json
import os

In [8]:
dir_path = os.getcwd() + "\\trait-layers"
items = {}
category_list = []
arr = []
for category in os.listdir(dir_path):
    category_path = os.path.join(dir_path,category)
    category_list.append(category)
    for item in os.listdir(category_path):
        arr.append(item)
    items[category] = arr.copy()
    arr.clear()
print(items)

{'background': ['blue@3x.png', 'gray@3x.png', 'green@3x.png', 'mix@3x.png', 'orange@3x.png', 'white@3x.png', 'yellow@3x.png'], 'bottom': ['blackoveralls@3x.png', 'denimpants@3x.png', 'greenoveralls@3x.png', 'koreanpants@3x.png', 'scoutpants@3x.png'], 'character': ['cat@3x.png', 'penguin@3x.png', 'puppy@3x.png', 'rabbit@3x.png', 'turtle@3x.png'], 'hair': ['glasses@3x.png', 'koreanhat@3x.png', 'ribbon@3x.png', 'sunglassees@3x.png'], 'hand': ['coke@3x.png', 'letter@3x.png', 'pencil@3x.png', 'phone@3x.png'], 'shoes': ['blackshoes@3x.png', 'pinkshoes@3x.png', 'rainboots@3x.png'], 'top': ['anipaltop@3x.png', 'bluetop@3x.png', 'koreangown_1@3x.png', 'scouttop@3x.png', 'startop@3x.png', 'stripetop@3x.png']}


In [9]:
## Generate Traits

TOTAL_IMAGES = 5400 # Number of random unique images we want to generate

all_images = [] 

def getWeights(items):
    weightsFor3 = [33,33,34]
    weightsFor4 = [25,25,25,25]
    weightsFor5 = [20,20,20,20,20]
    weightsFor6 = [16,16,17,17,17,17]
    weightsFor7 = [14,14,14,14,14,15,15]
    if len(items) == 3:
        return weightsFor3
    elif len(items) == 4:
        return weightsFor4
    elif len(items) == 5:
        return weightsFor5
    elif len(items) == 6:
        return weightsFor6
    elif len(items) == 7:
        return weightsFor7

# A recursive function to generate unique image combinations
def create_new_image():
    
    new_image = {} #

    # For each trait category, select a random trait based on the weightings 
    for i in range(len(category_list)):
        c = category_list[i]
        new_image [c] = random.choices(items[c], getWeights(items[c]))[0]
    
    if new_image in all_images:
        return create_new_image()
    else:
        return new_image
    
    
# Generate the unique combinations based on trait weightings
for i in range(TOTAL_IMAGES): 
    
    new_trait_image = create_new_image()
    
    all_images.append(new_trait_image)
    


In [10]:
# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

Are all images unique? True


In [11]:
# Add token Id to each image
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [12]:
#### Generate Metadata for all Traits 
METADATA_FILE_NAME = './metadata/all-traits.json'; 
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images, outfile, indent=4)

In [13]:
  
    
#### Generate Images    
for item in all_images:

    itemImgs = []
    for i in range(len(category_list)):
        c = category_list[i]
        itemImgs.append(Image.open(f'./trait-layers/{c}/{item[c]}').convert('RGBA'))

    #Create each composite
    comp = Image.alpha_composite(itemImgs[0], itemImgs[1])
    for i in range(1, len(category_list) - 1):
        comp = Image.alpha_composite(comp, itemImgs[i+1])

    #Convert to RGB
    rgb_im = comp.convert('RGB')
    file_name = str(item["tokenId"]) + ".png"
    rgb_im.save("./images/" + file_name)
    
    
    

In [ ]:
#### Generate Metadata for each Image    

f = open('./metadata/all-traits.json',) 
data = json.load(f)


IMAGES_BASE_URI = "ADD_IMAGES_BASE_URI_HERE"
PROJECT_NAME = "ADD_PROJECT_NAME_HERE"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for i in data:
    token_id = i['tokenId']
    token = {
        "image": IMAGES_BASE_URI + str(token_id) + '.png',
        "tokenId": token_id,
        "name": PROJECT_NAME + ' ' + str(token_id),
        "attributes": []
    }
    token["attributes"].append(getAttribute("Background", i["Background"]))
    token["attributes"].append(getAttribute("Circle", i["Circle"]))
    token["attributes"].append(getAttribute("Square", i["Square"]))

    with open('./metadata/' + str(token_id), 'w') as outfile:
        json.dump(token, outfile, indent=4)
f.close()